In [7]:
using Agents, Random, Distributions, PyPlot

Random.seed!(123);

In [8]:
mutable struct Human <: AbstractAgent
           id :: Int 
       status :: Symbol # :susceptible, :infected, or :removed
    infection :: Float64
end

agent_properties = [:status, :infection]

2-element Array{Symbol,1}:
 :status   
 :infection

In [9]:
function SIRModel(;
                              humans = 1000,         # Number of susceptible
                            infected = 10,           # Number of infected
                        contact_rate = 6.0,          # people / day
                      infection_rate = 0.01,         # probability of infection upon contact
                           time_step = 1/24          # days
                  )   

    model_properties = Dict(
                            :contact_rate   => contact_rate,
                            :infection_rate => infection_rate,
                            :humans         => humans,
                            :infected       => infected,
                            :p_infection    => 0.0,
                            :time_step      => time_step,
                           )   

    model = ABM(Human, scheduler=fastest, properties=model_properties)

    # Initialize susceptible population
    susceptible = humans - infected

    for i = 1:susceptible
        add_agent!(model, :susceptible, 0.0)
    end 

    # Initialize infected population
    for i = 1:infected
        add_agent!(model, :infected, 0)
    end 

    return model
end

model = SIRModel(); # test whether function works...

In [10]:
function agent_step!(agent, m)
    
    if agent.status == :removed
        return nothing
        
    elseif agent.status  == :susceptible
        
        # Define a stochastic process that determines whether the 
        # agent becomes infected
        
        # The (random) number of infected people this agent contacts
        infected_fraction = m.infected / m.humans   
        contacted = rand(Poisson(m.contact_rate * m.time_step * infected_fraction))
        
        # The probability that a susceptible is infected...
        p_infected =  1 - (1 - m.infection_rate)^contacted
        
        if rand(Bernoulli(p_infected))
            agent.status = :infected
        end
        
    elseif agent.status == :infected
        
        agent.infection += m.time_step
        
        # agent may become removed
    end 

    return nothing
end

function model_step!(model)
    infected_agents = [x.status === :infected for x in values(model.agents)]
    model.properties[:infected] = sum(infected_agents)
    return nothing
end

function count_infected(data, step)
    infected = filter(x -> (x.step == step && x.status === :infected), data) 
    return size(infected, 1)
end

function count_infected(data)
    infected = filter(x -> x.status === :infected, data) 
    return size(infected, 1)
end

count_infected (generic function with 2 methods)

In [16]:
function sir_simulation(; steps=30*24, kwargs...)

    model = SIRModel(; kwargs...)
    infected = zeros(steps)
    time = zeros(steps)

    for i = 1:steps
        step_data = step!(model, agent_step!, model_step!, 1, agent_properties)
        infected[i] = count_infected(step_data)
        time[i] = i * model.time_step
    end
    
    return time, infected
end

sir_simulation (generic function with 1 method)

In [ ]:
for i = 1:3
    for initial_infected in (1, 10, 100)
        time, infected_timeseries = sir_simulation(humans = 2000,   # number of people
                                                   infected = initial_infected,    # initial number of infected 
                                                   contact_rate = 6,      # per day
                                                   infection_rate = 0.01,   # probability of infection during contact
                                                   time_step = 1/24,   # time-step
                                                   steps = 30 * 24
                                                   )

        semilogy(time, infected_timeseries / infected_timeseries[1])
        xlabel("time (days)")
        ylabel("Pandemic / Initial infection size")
    end
end